# Part VI — Async, Realtime, and Background Work  
## 28. Realtime with WebSockets (Django Channels) — Notifications Done Right

This chapter adds **true realtime** to your Django project using **Django Channels**
(WebSockets). We’ll build an **organization-scoped notifications stream** so that
when something happens (e.g., a task status changes), connected browsers receive a
message instantly—no refresh needed.

Based on current upstream releases:
- `channels` latest stable is **4.3.2** (released Nov 20, 2025) and it lists
  compatibility up through Django **6.0** on PyPI.
- `channels_redis` is the official Redis-backed channel layer package.

(You don’t have to pin exactly these versions, but do pin compatible versions in
real projects.)

---

## 28.0 Learning Outcomes

By the end of this chapter you will be able to:

1. Explain Channels architecture: **ASGI**, **Consumers**, **Routing**, **Channel
   Layer**, **Groups**.
2. Configure Django + Channels to serve:
   - HTTP as normal
   - WebSocket connections under `/ws/...`
3. Authenticate WebSockets using Django sessions (cookie-based auth).
4. Prevent cross-site WebSocket abuse using **Origin validators** (Channels security).
5. Build an **OrgNotificationsConsumer** that:
   - requires login
   - requires org membership
   - joins an org “group”
   - sends JSON events to clients
6. Broadcast events from normal Django code (services) into the channel layer.
7. Run everything under an ASGI server (Uvicorn) locally.
8. Write automated tests for WebSockets using `WebsocketCommunicator`.

---

## 28.1 Why WebSockets (and when you should NOT use them)

### 28.1.1 What WebSockets give you
WebSockets create a long-lived connection:
- browser opens one connection
- server can push messages any time
- client can send messages without new HTTP requests

This enables:
- live notifications
- chat
- presence/typing indicators
- realtime dashboards
- collaborative editing (harder)

### 28.1.2 Alternatives and tradeoffs
**Polling** (HTTP GET every N seconds)
- simple
- wastes requests when nothing changes
- latency depends on interval

**SSE** (Server-Sent Events)
- server → client only (no client push)
- simpler than WebSockets for “stream updates”
- not as universal as HTTP, but good for many realtime feeds

**WebSockets**
- full duplex
- more moving parts (ASGI server, channel layer, scaling)
- requires careful security (Origin restrictions)

**Rule of thumb:**
- Use WebSockets when you need frequent near-instant updates or bidirectional comms.
- Don’t add WebSockets “because realtime is cool” if polling is enough.

---

## 28.2 Channels Architecture (The Real Mental Model)

Channels adds a new layer to Django:

### 28.2.1 Consumers
A **Consumer** is like a view, but for event-based protocols (WebSockets).
It handles events like:
- connect
- receive message
- disconnect
- messages sent via the channel layer

### 28.2.2 Routing
Separate from `urls.py`. WebSockets use **ASGI routing**, typically matching paths
like:

- `/ws/orgs/acme/notifications/`

### 28.2.3 Channel Layer
A channel layer is a message bus between:
- different consumer instances
- different server processes
- different machines

In production, it’s usually **Redis**.

### 28.2.4 Groups
A **group** is a broadcast target.
You add each connected socket to a group:

- `org-<org_id>`

Then you can broadcast:

- “Task status changed” → everyone connected to that org group receives it.

---

## 28.3 Install Channels + Redis Layer

### 28.3.1 Install packages
Add to `requirements.txt`:

```text
channels==4.3.2
channels-redis
```

Install:

```bash
python -m pip install -r requirements.txt
python -m pip freeze > requirements.txt
```

Notes:
- `channels-redis` is the Redis channel layer backend.
- You also need a Redis server (we’ll run one locally with Docker below).

### 28.3.2 Add `channels` to `INSTALLED_APPS`
In `config/settings.py` (or `config/settings/base.py`):

```python
INSTALLED_APPS = [
    # ...
    "channels",
    # your apps...
]
```

### 28.3.3 Set `ASGI_APPLICATION`
Add:

```python
ASGI_APPLICATION = "config.asgi.application"
```

This tells Channels which ASGI application object to use.

---

## 28.4 Configure CHANNEL_LAYERS (Dev and Production Reality)

### 28.4.1 Production-like (Redis) channel layer (recommended)
In settings:

```python
CHANNEL_LAYERS = {
    "default": {
        "BACKEND": "channels_redis.core.RedisChannelLayer",
        "CONFIG": {
            "hosts": [("127.0.0.1", 6379)],
        },
    },
}
```

**Why Redis**
- works across processes and machines
- supports groups at scale
- standard production setup

### 28.4.2 Dev-only in-memory layer (only for single-process learning)
You can do:

```python
CHANNEL_LAYERS = {
    "default": {
        "BACKEND": "channels.layers.InMemoryChannelLayer",
    }
}
```

But:
- it does not work if you run multiple workers/processes
- it will not scale
- messages won’t cross process boundaries

For this workbook, use Redis even in dev if you can.

---

## 28.5 Run Redis Locally (Docker)

If you have Docker:

```bash
docker run --rm -p 6379:6379 redis:7-alpine
```

Leave it running in a terminal tab.

---

## 28.6 Add WebSocket Routing to Your ASGI App (Critical Step)

Right now your `config/asgi.py` likely looks like “HTTP only.” We must upgrade it to
route both `http` and `websocket`.

### 28.6.1 Create a Realtime App (clean separation)
```bash
python manage.py startapp realtime
```

Add it to `INSTALLED_APPS`:

```python
INSTALLED_APPS += ["realtime"]
```

### 28.6.2 Create `realtime/routing.py`
```python
from django.urls import re_path

from realtime.consumers import OrgNotificationsConsumer

websocket_urlpatterns = [
    re_path(
        r"^ws/orgs/(?P<org_slug>[-\w]+)/notifications/$",
        OrgNotificationsConsumer.as_asgi(),
    ),
]
```

Explanation:
- We use `re_path` because WebSocket routing commonly uses regex patterns.
- `org_slug` is captured from the URL and passed in `scope["url_route"]["kwargs"]`.

### 28.6.3 Create `config/routing.py`
```python
from realtime.routing import websocket_urlpatterns

__all__ = ["websocket_urlpatterns"]
```

This keeps ASGI imports clean and avoids circular import surprises.

### 28.6.4 Update `config/asgi.py` to support websockets safely
Replace your ASGI file with this pattern:

```python
import os

from channels.auth import AuthMiddlewareStack
from channels.routing import ProtocolTypeRouter, URLRouter
from channels.security.websocket import AllowedHostsOriginValidator
from django.core.asgi import get_asgi_application

import config.routing

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "config.settings")

django_asgi_app = get_asgi_application()

application = ProtocolTypeRouter(
    {
        "http": django_asgi_app,
        "websocket": AllowedHostsOriginValidator(
            AuthMiddlewareStack(
                URLRouter(config.routing.websocket_urlpatterns),
            )
        ),
    }
)
```

#### Why these wrappers matter (security + auth)

- `AuthMiddlewareStack(...)`:
  - reads Django session cookie
  - sets `scope["user"]` like `request.user` in HTTP
- `AllowedHostsOriginValidator(...)`:
  - protects against cross-site WebSocket connection attempts
  - uses `ALLOWED_HOSTS` concept similarly to HTTP host validation
  - Channels docs explicitly recommend Origin validation for private sockets

Without Origin validation, any malicious site can attempt to open a socket to your
domain, and the browser will attach the user’s cookies. If your socket sends
private data, that’s a serious risk.

---

## 28.7 Build the Consumer (Org Notifications Stream)

We’ll implement an async JSON consumer:
- requires authenticated user
- requires membership in the org in the URL
- joins group `org-<org_id>`
- sends a welcome message
- receives broadcast events and forwards them to the client

### 28.7.1 Create `realtime/consumers.py`
```python
from __future__ import annotations

from channels.db import database_sync_to_async
from channels.generic.websocket import AsyncJsonWebsocketConsumer

from orgs.models import Membership, Organization


class OrgNotificationsConsumer(AsyncJsonWebsocketConsumer):
    """
    WebSocket consumer that streams realtime org-scoped notifications.

    URL:
      /ws/orgs/<org_slug>/notifications/

    Auth:
      Uses Django session auth via AuthMiddlewareStack, so scope["user"] works.

    Authorization:
      Only org members can connect.
    """

    async def connect(self):
        self.org_slug = self.scope["url_route"]["kwargs"]["org_slug"]
        user = self.scope["user"]

        if not user.is_authenticated:
            # 4401 is a common "unauthorized" close code used by some realtime APIs.
            await self.close(code=4401)
            return

        org = await self._get_org_by_slug(self.org_slug)
        if org is None:
            # We return 4404 to avoid leaking org existence (similar policy to 404).
            await self.close(code=4404)
            return

        is_member = await self._is_member(user_id=user.id, org_id=org.id)
        if not is_member:
            await self.close(code=4403)
            return

        self.org_id = org.id
        self.group_name = f"org-{self.org_id}"

        await self.channel_layer.group_add(self.group_name, self.channel_name)
        await self.accept()

        await self.send_json(
            {
                "type": "welcome",
                "org": {"id": self.org_id, "slug": self.org_slug},
            }
        )

    async def disconnect(self, close_code):
        # If connect() failed before org_id/group_name set, skip cleanup.
        group_name = getattr(self, "group_name", None)
        if group_name:
            await self.channel_layer.group_discard(group_name, self.channel_name)

    async def receive_json(self, content, **kwargs):
        """
        Optional: handle client->server messages.
        For notifications, we typically only need server->client pushes.

        We'll implement a simple ping/pong so you can test interactive messages.
        """
        msg_type = content.get("type")

        if msg_type == "ping":
            await self.send_json({"type": "pong"})
            return

        await self.send_json(
            {
                "type": "error",
                "error": {"code": "unknown_message", "message": "Unknown message type."},
            }
        )

    async def org_event(self, event):
        """
        Handler for group_send events of type "org.event".

        Channels maps event["type"] "org.event" -> method name "org_event".
        """
        payload = event.get("payload", {})
        await self.send_json(payload)

    @database_sync_to_async
    def _get_org_by_slug(self, slug: str) -> Organization | None:
        try:
            return Organization.objects.get(slug=slug)
        except Organization.DoesNotExist:
            return None

    @database_sync_to_async
    def _is_member(self, *, user_id: int, org_id: int) -> bool:
        return Membership.objects.filter(
            user_id=user_id,
            organization_id=org_id,
        ).exists()
```

### 28.7.2 Why `database_sync_to_async` is required
Consumers are async. Django ORM is sync-first and Channels requires you to wrap ORM
work with `database_sync_to_async` so it runs safely.

If you call ORM directly in async consumer methods, you risk:
- `SynchronousOnlyOperation`
- blocking the event loop
- poor concurrency under load

---

## 28.8 Broadcasting Events from Normal Django Code (Services → WebSocket)

Now we need to send messages to the org group when something happens in the app.

### 28.8.1 Create `realtime/broadcast.py`
```python
from __future__ import annotations

from asgiref.sync import async_to_sync
from channels.layers import get_channel_layer


def org_group_name(org_id: int) -> str:
    return f"org-{org_id}"


def broadcast_org_event(*, org_id: int, payload: dict) -> None:
    """
    Sends a payload to all websocket connections listening to the org group.

    Safe to call from sync code (views/services) because it uses async_to_sync.
    """
    channel_layer = get_channel_layer()
    if channel_layer is None:
        return

    async_to_sync(channel_layer.group_send)(
        org_group_name(org_id),
        {
            "type": "org.event",
            "payload": payload,
        },
    )
```

### 28.8.2 Why `async_to_sync` is needed
Most of your Django services and views are synchronous. `group_send` is async.
`async_to_sync` bridges sync code to the async channel layer correctly.

---

## 28.9 Wire Broadcasting into Your Task Workflow (Real Feature)

We already have TaskEvent creation in `tasks/services.py` (from Project 2 / Chapter
22). We will broadcast a notification whenever an event is created.

### 28.9.1 Update `tasks/services.py` (after creating TaskEvent)
In the places where you create `TaskEvent`, add:

```python
from realtime.broadcast import broadcast_org_event
```

Then after creating the event, broadcast it:

```python
event = TaskEvent.objects.create(
    task=task,
    actor=actor,
    action="status_changed",
    details={"from": old_status, "to": task.status},
)

broadcast_org_event(
    org_id=task.organization_id,
    payload={
        "type": "task_event",
        "org_id": task.organization_id,
        "task_id": task.id,
        "action": event.action,
        "actor_id": actor.id,
        "details": event.details,
        "created_at": event.created_at.isoformat(),
    },
)
```

Do similarly for:
- `created`
- `assignee_changed`
- etc.

### 28.9.2 Important design note: WebSocket delivery is “best effort”
Channel layer messages are typically **at-most-once** delivery:
- if the user is disconnected, they miss it
- if Redis is down, it may fail

That’s why we already store TaskEvent rows in the DB:
- DB is your source of truth
- WebSockets are a realtime convenience layer

If you need guaranteed delivery, you implement:
- a persistent notifications table + “unread” tracking
- and clients fetch missed notifications on reconnect

---

## 28.10 Add a Client-Side WebSocket (Browser) to See Notifications

We’ll add a small JS client on the tasks list page.

### 28.10.1 Create `static/js/org_notifications.js`
```javascript
(function () {
  function getWsScheme() {
    return window.location.protocol === "https:" ? "wss" : "ws";
  }

  function connect(orgSlug) {
    const wsScheme = getWsScheme();
    const wsUrl = `${wsScheme}://${window.location.host}` +
      `/ws/orgs/${orgSlug}/notifications/`;

    let socket = null;
    let retries = 0;

    function logLine(text) {
      const list = document.getElementById("ws-log");
      if (!list) return;

      const li = document.createElement("li");
      li.textContent = text;
      list.prepend(li);
    }

    function open() {
      socket = new WebSocket(wsUrl);

      socket.onopen = function () {
        retries = 0;
        logLine("WebSocket connected.");
        socket.send(JSON.stringify({ type: "ping" }));
      };

      socket.onmessage = function (event) {
        try {
          const msg = JSON.parse(event.data);

          if (msg.type === "welcome") {
            logLine(`Welcome to org ${msg.org.slug} notifications.`);
            return;
          }

          if (msg.type === "pong") {
            logLine("pong");
            return;
          }

          if (msg.type === "task_event") {
            logLine(
              `Task ${msg.task_id}: ${msg.action} ` +
                `details=${JSON.stringify(msg.details)}`
            );
            return;
          }

          logLine(`Message: ${event.data}`);
        } catch (e) {
          logLine(`Non-JSON message: ${event.data}`);
        }
      };

      socket.onclose = function () {
        logLine("WebSocket disconnected.");

        // Basic exponential backoff reconnect.
        retries += 1;
        const delay = Math.min(1000 * Math.pow(2, retries), 10000);
        window.setTimeout(open, delay);
      };

      socket.onerror = function () {
        // onerror is usually followed by onclose; keep it minimal.
      };
    }

    open();
  }

  const el = document.getElementById("realtime-org");
  if (!el) return;

  const orgSlug = el.dataset.orgSlug;
  if (!orgSlug) return;

  connect(orgSlug);
})();
```

### 28.10.2 Update `tasks/templates/tasks/list.html`
At the top of the content area (somewhere visible), add:

```django
{% load static %}

<div id="realtime-org" data-org-slug="{{ organization.slug }}"></div>

<h2>Realtime notifications</h2>
<ul id="ws-log"></ul>

<script src="{% static 'js/org_notifications.js' %}"></script>
```

Now:
1. Open `/orgs/acme/tasks/` in two browser tabs while logged in as a member.
2. Edit a task in one tab (change status).
3. Watch the other tab receive a `task_event` without refresh.

---

## 28.11 Running the Project (Use an ASGI Server)

### 28.11.1 Run under Uvicorn (recommended)
```bash
python -m uvicorn config.asgi:application --reload
```

This serves:
- HTTP
- WebSockets

Make sure Redis is running if you’re using `channels_redis`.

### 28.11.2 Common confusion: `python manage.py runserver`
Django’s default dev server is not a WebSocket server. Channels can provide a
WebSocket-capable runserver when Daphne is installed, but the simplest approach is:
- use Uvicorn for development once you add Channels

---

## 28.12 WebSocket Security (Non-Optional)

### 28.12.1 Origin validation (must do)
WebSockets can be initiated cross-site. If a socket sends private data and you do
not validate Origin, a malicious site could open a socket using the victim’s
cookies.

We already used:

- `AllowedHostsOriginValidator(...)`

You can also use `OriginValidator(...)` to be more explicit about allowed origins
(including scheme/port). Use that when you have multiple frontend domains.

### 28.12.2 Authorization: check membership before `accept()`
Do not accept and then “filter messages later.” That still exposes connection
metadata and can leak behavior.

Correct:
- validate user
- validate org membership
- join group
- accept

### 28.12.3 Data minimization in payloads
Only send what the client needs:
- task id, action, public fields
Avoid sending:
- emails
- internal notes
- anything sensitive unless the client truly needs it

---

## 28.13 Testing WebSockets (Automated, Repeatable)

Channels ships test utilities.

### 28.13.1 Install test dependency (if needed)
Channels provides testing tools; ensure Channels is installed. For safety, also
install `daphne` in dev if you encounter missing components:

```bash
python -m pip install daphne
```

(You can keep using Uvicorn for runtime; Daphne is often used in Channels examples
and sometimes in tests.)

### 28.13.2 Example test: member can connect and receive broadcast
Create `realtime/tests.py`:

```python
from __future__ import annotations

import asyncio

from asgiref.sync import async_to_sync
from channels.testing import WebsocketCommunicator
from django.contrib.auth import get_user_model
from django.test import TestCase, override_settings

from orgs.models import Membership, Organization
from realtime.broadcast import broadcast_org_event
from config.asgi import application


@override_settings(ALLOWED_HOSTS=["testserver", "localhost", "127.0.0.1"])
class RealtimeTests(TestCase):
    def _login_and_get_session_cookie(self, username: str, password: str) -> str:
        ok = self.client.login(username=username, password=password)
        self.assertTrue(ok)
        return self.client.cookies["sessionid"].value

    def test_member_receives_org_event(self):
        User = get_user_model()

        user = User.objects.create_user(
            username="u1",
            password="pass12345",
            email="u1@example.com",
        )
        org = Organization.objects.create(name="Acme", slug="acme")
        Membership.objects.create(
            organization=org,
            user=user,
            role=Membership.Role.MEMBER,
        )

        sessionid = self._login_and_get_session_cookie("u1", "pass12345")

        async def scenario():
            communicator = WebsocketCommunicator(
                application,
                "/ws/orgs/acme/notifications/",
                headers=[
                    (b"origin", b"http://testserver"),
                    (b"cookie", f"sessionid={sessionid}".encode("utf-8")),
                ],
            )

            connected, _ = await communicator.connect()
            self.assertTrue(connected)

            welcome = await communicator.receive_json_from()
            self.assertEqual(welcome["type"], "welcome")
            self.assertEqual(welcome["org"]["slug"], "acme")

            # Broadcast an event from sync land
            broadcast_org_event(
                org_id=org.id,
                payload={
                    "type": "task_event",
                    "task_id": 123,
                    "action": "status_changed",
                    "details": {"from": "open", "to": "done"},
                },
            )

            msg = await communicator.receive_json_from()
            self.assertEqual(msg["type"], "task_event")
            self.assertEqual(msg["task_id"], 123)

            await communicator.disconnect()

        async_to_sync(scenario)()
```

#### Why the test is structured like this
- `WebsocketCommunicator` runs against your ASGI `application`.
- We authenticate by:
  - logging in with Django test client
  - extracting `sessionid` cookie
  - sending it in the WebSocket handshake headers
- We set an `Origin` header so Origin validator logic behaves like a browser.

---

## 28.14 Troubleshooting Guide (Most Common Failures)

### Problem A: “WebSocket HANDSHAKING / REJECT / DISCONNECT”
Common causes:
1. `ALLOWED_HOSTS` missing the host you’re using
2. Origin validator rejecting your Origin header
3. Routing mismatch (URL pattern doesn’t match)
4. Consumer not called with `.as_asgi()`

Checklist:
- Confirm your websocket URL matches routing regex exactly.
- Confirm `AllowedHostsOriginValidator` is present and `ALLOWED_HOSTS` includes:
  - `localhost`, `127.0.0.1`, and your domain(s)
- Confirm your consumer uses `.as_asgi()` in routing.
- Run under an ASGI server (Uvicorn), not the default Django dev server.

### Problem B: Events are not delivered across tabs/processes
Cause:
- using InMemoryChannelLayer and multiple processes
Fix:
- use Redis channel layer (`channels_redis`)
- ensure all server instances share the same Redis

### Problem C: “RuntimeError: You cannot use the database_sync_to_async wrapper in the same thread”
Cause:
- incorrect mixing of event loops/threads (rare but happens)
Fix:
- ensure you’re using async consumer properly
- keep ORM calls inside `database_sync_to_async` wrappers
- avoid calling ORM in module import time

---

## 28.15 Exercises (Do These Before Continuing)

1. Add a second group stream:
   - `/ws/users/me/notifications/`
   - group name `user-<user_id>`
   - broadcast an event to just one user

2. Add “missed notifications” fallback:
   - create a `Notification` model stored in DB
   - on connect, send last 10 notifications to the client
   - then continue realtime streaming
   - explain why DB persistence matters

3. Add rate limiting behavior (basic):
   - if client sends > 5 messages/min (ping spam), close the socket
   - explain why application-level limits help

4. Write a test that:
   - non-member cannot connect (expect `connected == False` or close code 4403/4404)

---

## 28.16 Chapter Summary

- Channels adds WebSockets to Django via ASGI consumers and routing.
- The channel layer (Redis) enables cross-process broadcast and groups.
- Secure WebSockets by:
  - validating Origin (`AllowedHostsOriginValidator` / `OriginValidator`)
  - enforcing auth + membership before accept
- WebSocket notifications should be treated as “best effort”; persist important
  events in the DB if clients must not miss them.
- You can test WebSockets with `WebsocketCommunicator` and real session cookies.

---

Next chapter: **29. Background Tasks (Celery/RQ Concepts)**  
We’ll implement background jobs (sending emails, CSV export generation, report
building), retries, idempotency, scheduling, and a production-safe “task queue”
architecture that complements your realtime system.